# Query Transformations
Query transformations are a set of approaches focused on re-writing and / or modifying questions for retrieval.

![Alt text](query_transform.png)


In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")

# Part 1: Multi Query

![Alt text](multi_query.png)




Docs:

https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever


### Index

In [4]:
#### INDEXING ####

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
vectorstore = FAISS.from_documents(documents=splits, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\HAROON TRADERS\AppData\Local\Temp\ipykernel_14636\4106349506.py:34: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceBgeEmbeddings(


### Multi Query Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Multi Query: One Question, Multiple Perspectives.

template="""
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""


prompt_perspectives = ChatPromptTemplate.from_template(template)




generate_queries=(prompt_perspectives | ChatGroq(model="openai/gpt-oss-120b", temperature=0)|StrOutputParser()| (lambda x: x.split("\n")))

In [23]:
generate_queries.invoke("What is Task Decomposition")

['What does the term “task decomposition” mean?  ',
 'How is task decomposition defined in the context of problem solving?  ',
 'Can you explain the concept of task decomposition?  ',
 'What are the principles and steps involved in task decomposition?  ',
 'In what ways is a complex task broken down through task decomposition?']

In [24]:
# Load Unique documents only to avoid duplicates to be retrieved

from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]

    # Get unique documents
    unique_docs = list(set(flattened_docs))

    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

8

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:{context}

Question: {question}

"""


prompt = ChatPromptTemplate.from_template(template)

llm= ChatGroq(model="openai/gpt-oss-120b", temperature=0)

rag_chain= ({"context":retrieval_chain,"question":itemgetter("question")} | prompt | llm  | StrOutputParser())


rag_chain.invoke({"question":question})


'**Task decomposition** is the process by which an LLM‑powered autonomous agent breaks a large, complex request into a set of smaller, manageable sub‑goals or sub‑tasks that can be tackled one‑by‑one.  \n\nIn the agent architecture it belongs to the **Planning** component:\n\n* The agent first **identifies the sub‑goals** (e.g., “What are the steps for X?” or “List the sub‑tasks needed to achieve X?”).  \n* It then **orders or structures** those sub‑goals, often using prompting techniques such as **Chain‑of‑Thought (CoT)** – “think step‑by‑step” – or the more advanced **Tree‑of‑Thoughts**, which generates multiple possible thoughts at each step and searches the resulting tree (BFS/DFS) to pick the best path.  \n* Decomposition can be performed automatically by the LLM with simple prompts (“Steps for XYZ.\u202f1.”), with task‑specific instructions (e.g., “Write a story outline”), or with human input when needed.\n\nThe purpose of task decomposition is to let the LLM handle complex probl

# Part 2: RAG-Fusion

![Alt text](RAG_Fusion.png)


### Prompt

In [26]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

generate_queries = (
    prompt_rag_fusion 
    | ChatGroq(model="openai/gpt-oss-20b", temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [33]:
generate_queries.invoke("What is Task Decomposition for LLM agents?")

['1. “Task decomposition in large language model agents: definition and key concepts”  ',
 '2. “How do LLM agents perform task decomposition? Step‑by‑step examples”  ',
 '3. “Benefits of task decomposition for autonomous LLM agents”  ',
 '4. “Implementing task decomposition in LLM‑based agents: code snippets and best practices”']

# RRF Explanation

## Question Rankings
- **Question A**  
  1. Doc1  
  2. Doc4  
  3. Doc3  
  4. Doc2  

- **Question B**  
  1. Doc3  
  2. Doc1  
  3. Doc2  
  4. Doc4  

- **Question C**  
  1. Doc4  
  2. Doc3  
  3. Doc1  
  4. Doc2  

---

## Rank Positions
- **Doc1**  
  - Question A rank: **1**  
  - Question B rank: **2**  
  - Question C rank: **3**  

- **Doc2**  
  - Question A rank: **4**  
  - Question B rank: **3**  
  - Question C rank: **4**  

- **Doc3**  
  - Question A rank: **3**  
  - Question B rank: **1**  
  - Question C rank: **2**  

- **Doc4**  
  - Question A rank: **2**  
  - Question B rank: **4**  
  - Question C rank: **1**  

---

## Reciprocal Rank Fusion Calculation

![Alt text](RRF.png)


---

## Final Ranking
1. **Doc1** ≈ 0.0487  
2. **Doc3** ≈ 0.0487  
3. **Doc4** ≈ 0.0484  
4. **Doc2** ≈ 0.0469  

---

✅ **Conclusion**: Both **Doc1** and **Doc3** achieve the highest RRF scores, making them the most relevant across the question rankings.


In [34]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

9

In [35]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'**Task decomposition** is the step in an LLM‑powered autonomous agent where a big, complex request is broken down into a series of smaller, manageable sub‑goals or steps that the agent can execute one after another.  \n\nIn practice this is done by:\n\n* **Prompting the LLM** to “think step‑by‑step” (Chain‑of‑Thought) or to generate multiple possible thoughts at each step (Tree‑of‑Thoughts), thereby turning the original problem into a sequence of simpler subtasks.  \n* **Explicitly asking the model** for a list of sub‑goals, e.g., “What are the subgoals for achieving\u202fX?”  \n* Using **task‑specific instructions** (e.g., “Write a story outline”) or **human‑provided guidance** to define the decomposition.\n\nThe resulting sub‑tasks are then fed to the planning, memory, and tool‑use components of the agent, enabling it to tackle the overall objective in a structured, incremental way.'